In [1]:
import iris
import os, sys
import warnings
import numpy as np
import datetime
# Set the global warning filter to ignore all warnings
warnings.simplefilter("ignore")


In [52]:
fc_times = np.arange(6, 174, 6)  # 6 hourly data
def make_mergable(cubes, date):
    for i, cube in enumerate(cubes):
        cube.cell_methods = ()
        if cube.coords("forecast_period"):  # If missing
            cube.remove_coord("forecast_period")
        forecast_period_coord = iris.coords.AuxCoord(
            fc_times[i], standard_name="forecast_period",
            units=f"hours since {date.strftime('%Y-%m-%d %H:00:00')}")
        cube.add_aux_coord(forecast_period_coord)

        if cube.coords("time"):  # If missing
            cube.remove_coord("time")

        forecast_time_coord = iris.coords.AuxCoord(
            fc_times[i], standard_name="time",
            units=f"hours since {date.strftime('%Y-%m-%d %H:00:00')}")
        cube.add_aux_coord(forecast_time_coord)

        if cube.coords("time"):
            print(f"Cube {i} forecast_period: {cube.coord('time').units}")
        else:
            print(f"Cube {i} has no time coordinate")
    return iris.cube.CubeList(cubes).merge_cube()

def read_forecasts(date, forecast_files, var):
    forecast_cubes = []
    for forecast_file in forecast_files:
        forecast_cube = iris.load_cube(forecast_file, var)
        if var == 'precipitation_amount':
            if len(forecast_cube.shape) == 3:
                forecast_cube = forecast_cube.collapsed('time', iris.analysis.MEAN)
        else:
            if len(forecast_cube.shape) == 4:
                forecast_cube = forecast_cube.collapsed('time', iris.analysis.MEAN)
        forecast_cubes.append(forecast_cube)

    return make_mergable(forecast_cubes, date)


In [53]:
def concat_analysis_forecast(date, analysis_cube, forecast_cube):

    print(analysis_cube.shape, forecast_cube.shape)
    concatenated_array = np.concatenate((analysis_cube.data, forecast_cube.data), axis=0)

    analysis_datetime_list = sorted([date + datetime.timedelta(hours=(i + 1) * 6)
                                     for i in range(-self.ntimes_analysis, 0)])

    forecast_datetime_list = sorted([date + datetime.timedelta(hours=(i + 1) * 6)
                                     for i in range(0, self.ntimes_forecast)])

    all_dates = analysis_datetime_list + forecast_datetime_list

    # Convert datetime values to Iris-compatible units
    time_units = 'hours since 1970-01-01 00:00:00'
    time_values = iris.util.cf_units.date2num(all_dates, time_units, calendar='gregorian')

    # Create time coordinate
    time_coord = iris.coords.DimCoord(time_values, standard_name='time', units=time_units)

    # build cube
    dim_coords_and_dims = [(time_coord, 0)]

    for coord in analysis_cube.dim_coords[1:]:
        dim_coords_and_dims.append((coord, analysis_cube.coord_dims(coord)[0]))

    print(concatenated_array.shape)

    concat_cube = iris.cube.Cube(
        concatenated_array,
        long_name=forecast_cube.long_name,
        units=forecast_cube.units,
        attributes=None,
        dim_coords_and_dims=dim_coords_and_dims
    )

    return concat_cube

In [54]:
analysis_combined_file = '/data/scratch/prince.xavier/SALMON/processed_SEA_data/analysis/eqwaves/precipitation_amount_analysis_20250104_00.nc'

In [55]:
forecast_files = ['/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T006.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T012.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T018.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T024.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T030.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T036.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T042.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T048.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T054.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T060.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T066.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T072.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T078.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T084.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T090.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T096.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T102.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T108.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T114.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T120.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T126.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T132.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T138.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T144.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T150.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T156.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T162.pp', '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T168.pp']

In [56]:
forecast_files

['/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T006.pp',
 '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T012.pp',
 '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T018.pp',
 '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T024.pp',
 '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T030.pp',
 '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T036.pp',
 '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T042.pp',
 '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T048.pp',
 '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T054.pp',
 '/data/scratch/prince.xavier/SALMON/raw_data/mogreps/mogreps_bucket/20250104/00/000/qg00T060.pp',
 '/data/sc

In [57]:
var = 'precipitation_amount'
date = datetime.datetime(2025, 1, 4, 0)

In [58]:
print(f'analysis_combined_file: {analysis_combined_file}')
analysis_cube = iris.load_cube(analysis_combined_file)
forecast_cube = read_forecasts(date, forecast_files, var)
forecast_cube

analysis_combined_file: /data/scratch/prince.xavier/SALMON/processed_SEA_data/analysis/eqwaves/precipitation_amount_analysis_20250104_00.nc
Cube 0 forecast_period: hours since 2025-01-04 00:00:00
Cube 1 forecast_period: hours since 2025-01-04 00:00:00
Cube 2 forecast_period: hours since 2025-01-04 00:00:00
Cube 3 forecast_period: hours since 2025-01-04 00:00:00
Cube 4 forecast_period: hours since 2025-01-04 00:00:00
Cube 5 forecast_period: hours since 2025-01-04 00:00:00
Cube 6 forecast_period: hours since 2025-01-04 00:00:00
Cube 7 forecast_period: hours since 2025-01-04 00:00:00
Cube 8 forecast_period: hours since 2025-01-04 00:00:00
Cube 9 forecast_period: hours since 2025-01-04 00:00:00
Cube 10 forecast_period: hours since 2025-01-04 00:00:00
Cube 11 forecast_period: hours since 2025-01-04 00:00:00
Cube 12 forecast_period: hours since 2025-01-04 00:00:00
Cube 13 forecast_period: hours since 2025-01-04 00:00:00
Cube 14 forecast_period: hours since 2025-01-04 00:00:00
Cube 15 forecas

<iris 'Cube' of precipitation_amount / (kg m-2) (time: 28; latitude: 960; longitude: 1280)>

In [59]:
concat_analysis_forecast(date, analysis_cube, forecast_cube)

(332, 49, 360) (28, 960, 1280)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 49 and the array at index 1 has size 960

In [123]:

forecast_cubes = []
for forecast_file in forecast_files:
    forecast_cube = iris.load_cube(forecast_file, 'x_wind')
    if len(forecast_cube.shape) == 4:
        forecast_cube = forecast_cube.collapsed('time', iris.analysis.MEAN)
    forecast_cubes.append(forecast_cube)

In [124]:
make_mergable(forecast_cubes, date)

Cube 0 forecast_period: hours since 2025-01-01 00:00:00
Cube 1 forecast_period: hours since 2025-01-01 00:00:00
Cube 2 forecast_period: hours since 2025-01-01 00:00:00
Cube 3 forecast_period: hours since 2025-01-01 00:00:00
Cube 4 forecast_period: hours since 2025-01-01 00:00:00
Cube 5 forecast_period: hours since 2025-01-01 00:00:00
Cube 6 forecast_period: hours since 2025-01-01 00:00:00
Cube 7 forecast_period: hours since 2025-01-01 00:00:00
Cube 8 forecast_period: hours since 2025-01-01 00:00:00
Cube 9 forecast_period: hours since 2025-01-01 00:00:00
Cube 10 forecast_period: hours since 2025-01-01 00:00:00
Cube 11 forecast_period: hours since 2025-01-01 00:00:00
Cube 12 forecast_period: hours since 2025-01-01 00:00:00
Cube 13 forecast_period: hours since 2025-01-01 00:00:00
Cube 14 forecast_period: hours since 2025-01-01 00:00:00
Cube 15 forecast_period: hours since 2025-01-01 00:00:00
Cube 16 forecast_period: hours since 2025-01-01 00:00:00
Cube 17 forecast_period: hours since 2025

<iris 'Cube' of x_wind / (m s-1) (time: 28; pressure: 2; latitude: 961; longitude: 1280)>

In [125]:
def make_mergable(cubes, date):
    for i, cube in enumerate(cubes):
        cube.cell_methods = ()
        if cube.coords("forecast_period"):  # If missing
            cube.remove_coord("forecast_period")
        forecast_period_coord = iris.coords.AuxCoord(
            np.array([(i+1)*6]), standard_name="forecast_period", units=f"hours since {date.strftime('%Y-%m-%d %H:00:00')}")
        cube.add_aux_coord(forecast_period_coord)
            
        if cube.coords("time"):  # If missing
            cube.remove_coord("time")
        
        forecast_time_coord = iris.coords.AuxCoord(
            np.array([(i+1)*6]), standard_name="time", units=f"hours since {date.strftime('%Y-%m-%d %H:00:00')}")
        cube.add_aux_coord(forecast_time_coord)
            
        if cube.coords("time"):
            print(f"Cube {i} forecast_period: {cube.coord('time').units}")
        else:
            print(f"Cube {i} has no time coordinate")
    return iris.cube.CubeList(cubes).merge_cube()

In [126]:
iris.cube.CubeList(forecast_cubes).merge_cube()

<iris 'Cube' of x_wind / (m s-1) (time: 28; pressure: 2; latitude: 961; longitude: 1280)>